# 3.2 - h2o

Voy a probar la herramienta de AutoML de [h2o](https://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/intro.html). Lo usaré para comprobar también mis transformaciones.

In [1]:
# librerias

import h2o

from h2o.automl import H2OAutoML

import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split as tts 

In [2]:
# inicializamos el servidor h2o

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_292"; OpenJDK Runtime Environment (AdoptOpenJDK)(build 1.8.0_292-b10); OpenJDK 64-Bit Server VM (AdoptOpenJDK)(build 25.292-b10, mixed mode)
  Starting server from /Users/iudh/miniforge3/envs/clase/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/tmpqrmu66bw
  JVM stdout: /var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/tmpqrmu66bw/h2o_iudh_started_from_python.out
  JVM stderr: /var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/tmpqrmu66bw/h2o_iudh_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_iudh_b48m3k
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


## a) Transformado con selección de características

In [3]:
COLS=['accommodates', 'air_conditioning', 'availability_30', 'availability_365', 'availability_60', 'availability_90',
      'bathrooms', 'bedrooms', 'beds', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
      'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'cleaning_fee',
      'dishwasher', 'extra_people', 'guests_included','latitude', 'longitude', 'maximum_nights', 'minimum_nights', 
      'number_of_reviews', 'number_of_reviews_ltm', 'room_type_private_room', 'room_type_shared_room', 'security_deposit',
      'price']

len(COLS)

27

In [4]:
# carga de datos
listings=pd.read_csv('../data/transform_data/listings_normal.csv', usecols=COLS)

listings=listings[(listings.price>=10) & (listings.price<=196)]  

# cambio en el tamaño del tipo de dato
for c in listings.select_dtypes(include='int'):
    listings[c]=pd.to_numeric(listings[c], downcast='integer')

for c in listings.select_dtypes(include='float'):
    listings[c]=pd.to_numeric(listings[c], downcast='float')

In [5]:
train, test=tts(listings)

train=h2o.H2OFrame(train)
test=h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
# separa X de y
X=[c for c in train.columns if c!='price']

y='price'

In [7]:
# inicia auto-machine-learning

automl=H2OAutoML(max_models=20, seed=42, max_runtime_secs=300, sort_metric='RMSE')

In [8]:
%%time

# entrena varios modelos
automl.train(x=X, y=y, training_frame=train, validation_frame=test)

AutoML progress: |
16:44:58.494: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 1.78 s, sys: 234 ms, total: 2.02 s
Wall time: 5min 2s


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_3_AutoML_1_20221106_164458


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    103                103                         153105                 8            8            8             21            180           113.466

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 253.8915631068215
RMSE: 15.933975119436502
MAE: 10.865124652762091
RMSLE: 0.2382145798331422
Mean Residual Deviance: 253.8915631068215

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 432.7078814556822
RMSE: 20.801631701760375
MAE: 14.409912747681938
RMSLE: 0.3008227659362939
Mean Residual Deviance: 432.7078814556822

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 484.44023459089794
RMSE: 22.01000305749406
MAE: 15.121357245463214
RMSLE: 0.3181405610777855
Mean Residual Deviance: 484.44023459089794

Cross-Validation Metrics Summary: 
                        mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
mae                     15.1228   0.362934    15.6246       15.385        14.882        14.7854       14.9371
mean_residual_deviance  484.514   22.8813     512.182       505.479       465.548       462.993       476.369
mse                     484.514   22.8813     512.182       505.479       465.548       462.993       476.369
r2                      0.67687   0.00982015  0.671598      0.662726      0.687835      0.680102      0.682087
residual_deviance       484.514   22.8813     512.182       505.479       465.548       462.993       476.369
rmse                    22.0068   0.518234    22.6314       22.4829       21.5766       21.5173       21.8259
rmsle                   0.318117  0.00345412  0.319406      0.320647      0.314805      0.314085      0.321643

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse       training_mae        training_deviance    validation_rmse     validation_mae      validation_deviance
---  -------------------  ----------  -----------------  ------------------  ------------------  -------------------  ------------------  ------------------  ---------------------
     2022-11-06 16:45:54  3.598 sec   0.0                38.72062239500118   30.744091126005     1499.2865986562667   38.3987998588606    30.339184065603405  1474.467830600833
     2022-11-06 16:45:54  3.651 sec   5.0                30.279163619963484  23.549078075235418  916.8277495245202    30.19054693204986   23.456901249631144  911.4691240563052
     2022-11-06 16:45:54  3.707 sec   10.0               26.0691124877713    19.7008886121055    679.5986259000736    26.3698444501494    19.88795309662943   695.3686963250752
     2022-11-06 16:45:54  3.759 sec   15.0               23.496132673937893  17.283851248297015  552.0682506312918    24.245160360001265  17.792015393234657  587.8278008821766
     2022-11-06 16:45:54  3.808 sec   20.0               22.161536564323924  15.976813645786574  491.1337028918662    23.287148581192906  16.768191623943597  542.2912890425548
     2022-11-06 16:45:54  3.858 sec   25.0               21.090366849646507  14.974461624579034  444.8035738526683    22.679189804839577  16.08371902418206   514.3456502039395
     2022-11-06 16:45:54  3.905 sec   30.0               20.388481127524283  14.320936360269881  415.6901626874138    22.352343202289866  15.70761281262198   499.62724663295404
     2022-11-06 16:45:54  3.952 sec   35.0               19.783130227452478  13.803083463850667  391.37224159634394   22.02513561747

In [9]:
print('Leaderboard:')

leader=automl.leaderboard

leader.head()

Leaderboard:


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_3_AutoML_1_20221106_164458,22.01,484.44,15.1214,0.318141,484.44
GBM_2_AutoML_1_20221106_164458,22.0947,488.178,15.2342,0.318675,488.178
GBM_5_AutoML_1_20221106_164458,22.1141,489.036,15.2886,0.320104,489.036
GBM_4_AutoML_1_20221106_164458,22.184,492.132,15.1937,0.319876,492.132
GBM_grid_1_AutoML_1_20221106_164458_model_2,22.392,501.401,15.39,0.324905,501.401
GBM_1_AutoML_1_20221106_164458,22.4125,502.321,15.5184,nan,502.321
GBM_grid_1_AutoML_1_20221106_164458_model_1,22.4758,505.162,15.6401,0.325074,505.162
XGBoost_3_AutoML_1_20221106_164458,22.4779,505.258,15.7033,0.33362,505.258
DRF_1_AutoML_1_20221106_164458,23.084,532.872,16.0003,0.335943,532.872
XRT_1_AutoML_1_20221106_164458,23.087,533.011,16.0471,0.336548,533.011


In [10]:
# evaliuación mejor modelo

automl.leader.model_performance(test)

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 432.7078814556822
RMSE: 20.801631701760375
MAE: 14.409912747681938
RMSLE: 0.3008227659362939
Mean Residual Deviance: 432.7078814556822

## b) Transformado sin selección de características

In [11]:
%%time

df_normal=h2o.import_file(path='../data/transform_data/listings_normal.csv')

normal_train, normal_test=df_normal.split_frame(ratios=[.8])

X=[c for c in normal_train.columns if c!='price']
y='price'

normal_automl=H2OAutoML(max_models=20, seed=42, max_runtime_secs=300, sort_metric='RMSE')

normal_automl.train(x=X, y=y, training_frame=normal_train, validation_frame=normal_test)

normal_leader=normal_automl.leaderboard

normal_leader.head()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
16:50:04.57: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
16:50:04.123: _train param, Dropping bad and constant columns: [shared_pool, property_type_igloo, ground_floor_access, steam_oven]

█████
16:50:25.494: _train param, Dropping bad and constant columns: [shared_pool, property_type_igloo, ground_floor_access, steam_oven]

██
16:50:35.854: _train param, Dropping bad and constant columns: [shared_pool, property_type_igloo, ground_floor_access, steam_oven]

█████████████
16:51:36.81: _train param, Dropping bad and constant columns: [shared_pool, property_type_igloo, ground_floor_access, steam_oven]

██
16:51:48.86: _train param, Dropping bad and constant columns: 

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_4_AutoML_2_20221106_165004,276.985,76720.8,75.776,nan,76720.8
GBM_2_AutoML_2_20221106_165004,281.008,78965.3,77.2932,nan,78965.3
GBM_3_AutoML_2_20221106_165004,281.807,79415.2,75.3565,nan,79415.2
DRF_1_AutoML_2_20221106_165004,284.099,80712.5,68.7857,0.626416,80712.5
GBM_1_AutoML_2_20221106_165004,285.633,81586.3,85.5763,nan,81586.3
XGBoost_3_AutoML_2_20221106_165004,286.045,81821.7,81.908,nan,81821.7
XGBoost_1_AutoML_2_20221106_165004,288.307,83120.8,102.973,nan,83120.8
XGBoost_2_AutoML_2_20221106_165004,291.53,84989.6,89.1768,nan,84989.6
GLM_1_AutoML_2_20221106_165004,330.16,109006,112.573,nan,109006


In [12]:
normal_automl.leader.model_performance(normal_test)

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 57526.06713609767
RMSE: 239.8459237429264
MAE: 71.99968555193226
RMSLE: NaN
Mean Residual Deviance: 57526.06713609767

## c) En crudo

In [13]:
%%time

df_raw=h2o.import_file(path='../data/clean_data/listings.csv')

raw_train, raw_test=df_raw.split_frame(ratios=[.8])

X=[c for c in raw_train.columns if c!='price']
y='price'

raw_automl=H2OAutoML(max_models=20, seed=42, max_runtime_secs=300, sort_metric='RMSE')

raw_automl.train(x=X, y=y, training_frame=raw_train, validation_frame=raw_test)

raw_leader=raw_automl.leaderboard

raw_leader.head()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
16:55:07.766: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████
16:55:57.709: GLM_1_AutoML_3_20221106_165507 [GLM def_1] failed: DistributedException from /127.0.0.1:54321: 'Java heap space', caused by java.lang.OutOfMemoryError: Java heap space

████████████████████████████████████████████████████| (done) 100%
CPU times: user 1.55 s, sys: 247 ms, total: 1.8 s
Wall time: 5min 4s


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_4_AutoML_3_20221106_165507,251.924,63465.8,73.1255,nan,63465.8
GBM_5_AutoML_3_20221106_165507,255.828,65447.7,69.9521,nan,65447.7
GBM_3_AutoML_3_20221106_165507,256.265,65671.9,73.882,nan,65671.9
GBM_2_AutoML_3_20221106_165507,256.629,65858.3,72.8398,nan,65858.3
XGBoost_3_AutoML_3_20221106_165507,259.697,67442.3,73.6438,nan,67442.3
XGBoost_1_AutoML_3_20221106_165507,263.039,69189.7,99.972,nan,69189.7
GBM_1_AutoML_3_20221106_165507,263.977,69684.1,90.5096,nan,69684.1
XRT_1_AutoML_3_20221106_165507,264.077,69736.6,71.753,0.681172,69736.6
DRF_1_AutoML_3_20221106_165507,267.724,71676.1,66.8459,0.615568,71676.1
XGBoost_2_AutoML_3_20221106_165507,271.1,73495.3,85.1296,nan,73495.3


In [14]:
raw_automl.leader.model_performance(raw_test)

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 81612.10605564358
RMSE: 285.6783261916164
MAE: 72.67040872384361
RMSLE: NaN
Mean Residual Deviance: 81612.10605564358